
<br>
Created on 10-Sep-2021<br>
@author: sowjanya y<br>


In [ ]:
import pandas as pd
from itertools import permutations
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from itertools import combinations
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
def EXP_PRE():
    df = pd.read_csv("Employee_skills_traits.csv")
    
    #removing the ID column and writing remaining columns to a separate dataframe
    order = [1,2,3,4,5,6,7,8,9,10,11,12,13,] # setting column's 
    df_d = df[[df.columns[i] for i in order]]
    df_d.info() # to get information on the data
    df_d.describe().to_csv('Describe_dataframe.csv', index = True) #to get the statistics of the columns of dataframe
    #print("Number of rows and columns after removing the ID column :")
    #print( )
    #print(df_d)
    

rop the duplicate rows

In [ ]:
    df_d = df_d.drop_duplicates()
    #print("after removing duplicates: ")
    #print( )
    #print(df_d)

In [ ]:
 # IQR    
    Q1 = df_d.quantile(0.25)
    Q3 = df_d.quantile(0.75)
    IQR = Q3 - Q1
    #print(IQR)

    #removing noisy data 
    df_d = df_d[~((df_d < (Q1 - 1.5 * IQR)) |(df_d > (Q3 + 1.5 * IQR))).any(axis=1)]
    
    df_d.describe().to_csv('Describe_remove_noisy.csv', index = True) 

here Features - Age, Employment period and Time in current department columns <br>
are taken for normalization to handle values with varying magnitude

In [ ]:
    x = ['Employment period ','Time in current department ', 'Age ']
    #print ("\nOriginal data values : \n",  x)
    """ MIN MAX SCALER """
  
    min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1))
  
    # Scaled feature
    df_d[x] = min_max_scaler.fit_transform(df_d[x])
    #print ("\nAfter min max Scaling of the columns: \n", df_d[x])
    
   
    
    #split data into train data and test data
    train, test = train_test_split(df_d, test_size=0.2)
    train.describe().to_csv('Decribe_train.csv', index = True) 
    test.describe().to_csv('Describe_test.csv', index = True) 
    return train, test
    
#This function is used to get the frequent Pattern sets
def FREQ_PATTERN(data, min_support, max_length):
    
    si = data.shape[0] # get the number of rows
    # min_suppport is minimum threshold support.
    # where max_ Length is maximum number of Columns to be included in Pattern set.

    # Step 1:
# Creating a dictionary to stored support of an Pattern set.
   
    support = {}
    L = set(train.columns)
    #print("L is:", L)

    # Step 2:
    # Generating combination of Columns with L and ith iteration
    for i in range(1, max_length+1): 
        c = list(combinations(L,i))
        #print("Combi is:", c)

        # Reset "L" for next (i+l)th iteration
        L =set()
        # Step 3:
        #iterate through each item in "c"
        for j in list(c):
            sup = data.loc[:,j].product(axis=1).sum()/len(data.index)
            #print("sup is:", sup)
            if sup > min_support: 
                support[j]= sup
                # Appending frequent Pattern set in List "L ", already reset List "L"
                L = set(set(L) |  set(j))
                #print("L after supp:",L)
        
        #i = i+1
# Step 4: data frame with columns "Patterns", 'support'
    result = pd.DataFrame(list(support.items()), columns = ["Patterns" , "Support", ])
    if si == 798:
        if min_support == 0.25:
            result.to_csv('Freq_Columns_train_0.25S.csv', index = True)
        else:
            result.to_csv('Freq_Columns_train_0.30S.csv', index = True)
    else:
        if min_support == 0.21:
            result.to_csv('Freq_Columns_test_0.21S.csv', index = True)
        else:
            result.to_csv('Freq_Columns_test_0.35S.csv', index = True)
    #print(result)
    return(result)

his function is used to get the association rules

In [ ]:
def ASSOCIATION_RULE(df, min_threshold, min_support, fl):

STEP 1:<br>
reating required variable

In [ ]:
    support = pd.Series(df.Support.values, index=df.Patterns).to_dict() 
    data = []
    L= df.Patterns.values
    #print(L)

Step 2:<br>
enerating rule using permutation

In [ ]:
    p = list(permutations(L, 2))
    #print(p)

Iterating through each rule

In [ ]:
    for i in p:

If LHS(Antecedent)of rule is subset of RHS then valid rule.

In [ ]:
        if set(i[0]).issubset(i[1]):
            conf = support[i[1]]/support[i[0]]
            if conf > min_threshold :
                j = i [1][not i[1].index(i[0][0])]
                lift = support[i[1]]/(support[i[0]]* support[(j,)])
                data.append ([i[0], (j,), support[i[0]], support[(j,)], support[i[1]], conf, lift ])

STEP 3:

In [ ]:
    result = pd.DataFrame(data, columns = ["antecedents", "consequents", "antecedent support", 
                                            "consequent support", "support", "confidence", "Lift" ])
    
    if fl == 'train':
        if min_support == 0.25:
            result.to_csv('Rules_train_0.25S.csv', index = True)
        else:
            result.to_csv('Rules_train_0.30S.csv', index = True)
    else:
        if min_support == 0.21:
            result.to_csv('Rules_test_0.21S.csv', index = True)
        else:
            result.to_csv('Rules_test_0.35S.csv', index = True)
    #print(result)
    return (result)

In [ ]:
if __name__ == '__main__':
    train, test = EXP_PRE()
    #print(train.shape)
    #print(test.shape)
    
#finding frequent Pattern sets for train data when support is 0.25
    freq_patternset_train = FREQ_PATTERN(train, 0.25, 4)
    freq_patternset_train.sort_values(by = 'Support', ascending = False)
    # Rule with minimum confidence 50% for train data
    my_rule = ASSOCIATION_RULE(freq_patternset_train, 0.5, 0.25, 'train')
    
#finding frequent Pattern sets for train data when support is 0.3
    freq_patternset_train = FREQ_PATTERN(train, 0.3, 4)
    freq_patternset_train.sort_values(by = 'Support', ascending = False)
    # Rule with minimum confidence 50% for train data
    my_rule = ASSOCIATION_RULE(freq_patternset_train, 0.5, 0.3, 'train')
    
    #finding frequent Pattern sets for test data when support is 0.21
    freq_patternset_test= FREQ_PATTERN(test, 0.21, 4)
    freq_patternset_test.sort_values(by = 'Support', ascending = False)
    # Rule with minimum confidence 50% for test data
    my_rule = ASSOCIATION_RULE(freq_patternset_test, 0.5, 0.21, 'test')
    
#finding frequent Pattern sets for test data when support is 0.35
    freq_patternset_test= FREQ_PATTERN(test, 0.35, 4)
    freq_patternset_test.sort_values(by = 'Support', ascending = False)
    # Rule with minimum confidence 50% for test data
    my_rule = ASSOCIATION_RULE(freq_patternset_test, 0.5, 0.35, 'test')
    
    print("KDD process successful")